# Test CNN model with audio

In [16]:
# import libraries
import IPython.display as ipd
import librosa
import librosa.display
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
import numpy as np
import os
import sys
import struct
import pyaudio
import time
from keras.models import load_model

# Inputs 

In [52]:
# TRAINED MODEL
# model folder
MODEL_FP = 'models/'

# SAMPLE DATA
# audio file path
SAMPLE_AUDIO_FP = 'audio_sample/'

# sample audio spreadsheet name
SAMPLE_AUDIO_FN = 'list_of_sample_audio.xlsx'

# settings
tot_channels = 1
tot_rows = 40
max_pad_len = 174


# STREAM SETTINGS
# how many samples per frame
CHUNK = 1024 * 2
#FORMAT = pyaudio.paInt16
FORMAT = pyaudio.paFloat32
CHANNELS = 1
RATE = 44100 # samples per second, hertz

In [3]:
# define classes
# 0 = air_conditioner
# 1 = car_horn
# 2 = children_playing
# 3 = dog_bark
# 4 = drilling
# 5 = engine_idling
# 6 = gun_shot
# 7 = jackhammer
# 8 = siren
# 9 = street_music
    
class_label = ['air_conditioner', 'car_horn', 'children_playing', 'dog_bark',
          'drilling', 'engine_idling', 'gun_shot', 'jackhammer',
          'siren', 'street_music']

## Functions

In [51]:
  
# function used to extract feature from sample audio

def extract_features2(in_data):
    try:
        numpy_array = np.frombuffer(in_data, dtype=np.float32)
        mfccs =librosa.feature.mfcc(numpy_array, n_mfcc=tot_rows)
        pad_width = max_pad_len - mfccs.shape[1]
        mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
        
    except Exception as e:
        print("Error encountered while parsing file: ", in_data)
        return None 
     
    return mfccs

# Load model

In [5]:
# load model
model=load_model(MODEL_FP + 'cnn_basic_model.hdf5')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



# Predict from stream 

In [6]:
%matplotlib qt

In [53]:
# pyaudio object
p = pyaudio.PyAudio()

stream = p.open(
        format=FORMAT,
        channels=CHANNELS,
        rate=RATE,
        input=True,
        output=True,
        frames_per_buffer=CHUNK
        #stream_callback = callback
)


In [ ]:
fig, ax = plt.subplots()

i= 0
while True:
    data = stream.read(CHUNK)
    
    i= i + 1
    print(i)
    
    # feature
    feature = extract_features2(data)
    feature = feature.reshape(1, tot_rows, max_pad_len, tot_channels)
    
    # all class prediction probability
    pred_prob_vector = model.predict_proba(feature)
    pred_prob_class = pred_prob_vector[0]
    
    # clear 
    ax.clear()
    
    # set x limit
    ax.set_xlim(0,1)
    
    # plot
    bar = ax.barh(class_label,pred_prob_class)
    fig.canvas.draw()
    #plt.pause(0.001)
    fig.canvas.flush_events()
    
    feature = None
    pred_prob_class = None

#stream.close()
#p.terminate()

In [61]:
clear(pred_prob_class)